In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-03 23:16:12,970 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:16:12,971 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:16:12,972 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:16:12,976 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-03 23:16:12,976 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-03 23:16:12,987 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-03 23:16:13,221 - settings - Setting enable_python_native_blobs to True


In [3]:
test_mode = True

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-03 23:16:13,339 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:16:13,340 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:16:13,341 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:16:13,342 - settings - Setting enable_python_native_blobs to True


Sleeping 94 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-03 23:16:13,570 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [5]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 120


In [8]:
#so that it will have the adapter defined
from datajoint_utils import *

In [9]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (minnie.Decimation().proj(decimation_version='version')  & 
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
              & minnie.MultiSomaProofread() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity=2").proj()))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988402042,0,0.25
864691134988472442,0,0.25
864691135012400886,0,0.25
864691135081644023,0,0.25
864691135081658359,0,0.25
864691135103989069,0,0.25
864691135181738242,0,0.25
864691135181779714,0,0.25
864691135207768697,0,0.25
864691135210033344,0,0.25


In [10]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """

#     key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0" & "segment_id=864691136309663834").proj())
#     key_source = (minnie.Decimation() & "n_faces>500000").proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()

    
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity=2").proj()))

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [62]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988402042,0,0.25,=BLOB=,939750,1891203,1,0,1,2,11,258,129,10036082.322668254,3813311.2574745454,18993.006,98.476,98.476,483.731,464.87,3462,0.0003449553210798665,13.418604651162791,6704668.599,123,0.0005163566176136366,28.146,158216454358.305,22721587.50666046,15764.762511059254,23597.952981882347,1286312637.05939,3800.2828
864691134988472442,0,0.25,=BLOB=,760380,1540479,2,2,1,2,13,356,154,8946949.505293293,4539226.960667551,14612.677,83.572,83.572,460.773,357.467,2745,0.00030680848241917237,7.71067415730337,3989981.809,120,0.0006879730613829473,22.875,142610143158.787,29636743.193003334,15939.526994582277,35742.05346929365,1188417859.6565583,2946.8047
864691135012400886,0,0.25,=BLOB=,802480,1644474,1,1,1,2,16,267,95,7874984.993421403,2360030.914041459,18829.304,258.808,234.344,445.408,404.608,3802,0.0004827945708056728,14.239700374531836,6801293.033,192,0.0005590113499819264,19.802,237757391144.08643,35898024.06526593,30191.472281242943,34957.67495834736,1238319745.5421169,2740.7456
864691135081644023,0,0.25,=BLOB=,853714,1717788,3,2,2,2,12,274,133,9153647.235968873,3241695.1367920083,19845.782,148.853,147.244,444.527,417.096,3462,0.0003782098993717189,12.635036496350365,6075226.747,137,0.0005698552735845729,25.27,247942694346.279,43604637.473335266,27086.765302905555,40812.088942806164,1809800688.6589708,3245.9033
864691135103989069,0,0.25,=BLOB=,1245458,2502307,1,0,1,2,16,318,79,11735683.023595676,2465167.2976137036,22362.941,328.735,267.975,496.731,445.443,5680,0.0004839939855720229,17.861635220125788,8425505.074,202,0.0006741435617346826,28.119,303932625475.8736,27530848.576665558,25898.1624558868,36072.92652564767,1504616957.8013544,5048.7295
864691135181738242,0,0.25,=BLOB=,1254907,2542551,1,1,0,1,7,187,119,6130376.608708815,3973535.0751242815,20852.979,315.254,264.465,535.075,493.113,3739,0.0006099135891077843,19.994652406417114,5667370.363,153,0.0006597416015742397,24.438,169740974876.60693,23933860.84416135,27688.506875005503,29950.570371186262,1109418136.4484112,3144.3561
864691135181779714,0,0.25,=BLOB=,1165683,2363249,1,0,1,2,17,293,140,11275458.815910662,4181172.1191427135,18048.623,124.394,124.394,528.008,473.034,4684,0.0004154154679178523,15.986348122866895,7484517.424,14

In [63]:
curr_table

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,090c7be9245f8e5aeb0e40175d2cec86,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.155,at-node28,29,22130,2021-01-04 23:47:03
__decomposition,0b551f0304ab0afc29b5743c69c54930,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.131,at-node4,29,22119,2021-01-04 23:25:28
__decomposition,1225e50846062786a6ef422bce31d968,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.150,at-node23,29,22288,2021-01-04 23:45:49
__decomposition,15334694f16f59d3b363e63519c951d4,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.134,at-node7,29,22133,2021-01-04 23:40:45
__decomposition,1622bc6d61dd93b996d63e9c981f4462,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.144,at-node17,29,22117,2021-01-04 23:06:27
__decomposition,18720205cf210f8825988e024e22e1b0,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.139,at-node12,29,22129,2021-01-04 23:07:28
__decomposition,1a2b1e7bc755fb777b733d20db3b5c16,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.130,at-node3,29,22120,2021-01-04 21:57:45
__decomposition,1d9749840c8ef1781b47b8ca53c76624,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.159,at-node32,29,22116,2021-01-04 23:45:24
__decomposition,4932e4f47fa3a786c14b3214e5152f96,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.142,at-node15,29,22121,2021-01-04 22:16:52
__decomposition,4abdca51f85665bc2b7342fd50b29015,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.143,at-node16,29,22111,2021-01-04 23:07:42


In [53]:
schema.external['decomposition'].delete(delete_external_files=True)

100%|██████████| 29/29 [00:00<00:00, 66.57it/s]


[]

In [55]:
curr_table.delete()

In [58]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,1dabd9be0656c837a3baac4e2113b48b,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.130,at-node3,29,22120,2021-01-04 20:42:03
__decomposition,22efd0541a14a713e292d1e6da2e2962,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.169,at-node42,29,22100,2021-01-04 20:39:22
__decomposition,3a002ead65c7367cc6259695157121d4,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.139,at-node12,29,22129,2021-01-04 20:40:34
__decomposition,54646fec4005fb61b40b960eee14c9fc,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.171,at-node44,29,22108,2021-01-04 20:40:33
__decomposition,5ac976f47d9833f16e6829ea102c9a52,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.148,at-node21,29,22107,2021-01-04 20:38:46
__decomposition,8f19987e533dc40be06acb39c8ffa173,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.168,at-node41,29,22125,2021-01-04 20:41:38
__decomposition,b1ebe69c4b711d9a802e50b1c214a4d5,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.144,at-node17,29,22117,2021-01-04 20:41:31
__decomposition,b46c4e55b7ec91c35d4e7c084f9ef946,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.160,at-node33,29,22106,2021-01-04 20:40:34
__decomposition,d0749496eee10349c3a48b7048441d97,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.172,at-node45,29,22132,2021-01-04 20:42:37
__decomposition,e41d371a8feedcbcdfbe55e1eb84cf23,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.164,at-node37,29,22131,2021-01-04 20:41:20


In [22]:
curr_table.delete()

In [64]:
possible_keys = [
"faac03024aa088e0e4f204bca3f3962f",
"26f6cebdc9e967351f8d399546dcc588",
"0346046686f7bf941663a32a3d275407"
]

In [65]:
cur_error = (curr_table & dict(key_hash = possible_keys[0]))
cur_error

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,faac03024aa088e0e4f204bca3f3962f,error,=BLOB=,Exception: Something went wrong when trying to find end nodes,=BLOB=,celiib@10.28.0.157,at-node30,29,22102,2021-01-04 22:58:02


In [68]:
print(cur_error.fetch1("key"))

{'segment_id': 864691135946659937, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


In [66]:
print(cur_error.fetch1("error_stack"))

Traceback (most recent call last):
  File "/meshAfterParty/skeleton_utils.py", line 1775, in clean_skeleton
    end_node_must_keep_idx = np.where(end_nodes==end_node_idx)[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "Decomposition_for_multi_soma.py", line 223, in make
    widths_to_calculate=["no_spine_median_mesh_center"]
  File "/meshAfterParty/neuron.py", line 1771, in __init__
    use_meshafterparty=use_meshafterparty)
  File "/meshAfterParty/preprocessing_vp2.py", line 3045, in preprocess_neuron
    error_on_no_starting_coordinates=True
  File "/meshAfterParty/preprocessing_vp2.py", line 2580, in preprocess_limb
    error_on_no_starting_coordinates=error_on_no_starting_coordinates
  File "/meshAfterParty/preprocessing_vp2.py", line 832

In [ ]:
# import pandas as pd
# key_hash,error_message = curr_table.fetch("key_hash","error_message")

# df = pd.DataFrame.from_dict([dict(key_hash = k,error_message = m) for k,m in zip(key_hash,error_message)])
# df
# #df.columns = ["error","key_hash"]
# key_hashes_to_delete = df[df["error_message"].str.contains("OSError")]["key_hash"].to_numpy()

# (curr_table & [dict(key_hash=k) for k in key_hashes_to_delete]).delete()

In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

In [ ]:
debug